In [1]:
import torch
import csv
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

Load Model & Tokenizer

In [2]:
from transformers import BitsAndBytesConfig

model_id = "meta-llama/Llama-3.2-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading weights:   0%|          | 0/254 [00:00<?, ?it/s]

Data Formatting

In [3]:
def format_example(text, label=""):
    return f"""### Інструкція:
Класифікуй текст як один із класів:
Neutral, Hate-Speech, або Sarcasm.

### Текст:
{text}

### Відповідь:
{label}"""

Evaluation Logic

In [8]:
def predict(text):
    # We add a "System" style instruction to tell the model to look for sarcasm
    prompt = (
        "Класифікуй тональність повідомлення. "
        "Обери одну категорію: Hate-Speech, Sarcasm, або Neutral.\n\n"
        f"Текст: {text}\n"
        "Категорія:"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Use 'do_sample=False' to get the most likely answer (greedy decoding)
    outputs = model.generate(
        **inputs, max_new_tokens=5, do_sample=False, pad_token_id=tokenizer.eos_token_id
    )

    # We decode only the NEW tokens generated, not the whole prompt
    input_length = inputs.input_ids.shape[1]
    prediction_tokens = outputs[0][input_length:]
    decoded = tokenizer.decode(prediction_tokens, skip_special_tokens=True).strip()

    # Check the output against our labels
    for label in ["Hate-Speech", "Sarcasm", "Neutral"]:
        if label.lower() in decoded.lower():
            return label
    return "Neutral"

Result logging

In [9]:
def log_result(stage, text, gold, pred, path="results.csv"):
    header = ["stage", "text", "gold", "pred"]
    row = [stage, text, gold, pred]

    file_exists = False
    try:
        with open(path, "r", encoding="utf-8") as f:
            file_exists = True
    except FileNotFoundError:
        pass

    with open(path, "a", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(header)
        writer.writerow(row)

Zero-shot baseline

In [ ]:
test_texts = [
    ("Ти повний дурень!", "Hate-Speech"),
    ("Гарне відео!", "Neutral"),
    ("Ого, яка глибока думка 🤡", "Sarcasm"),
]

for text, gold in test_texts:
    pred = predict(text)
    log_result("zero-shot", text, gold, pred)
    print(f"Текст: {text}")
    print(f"Expected: {gold} | Predicted: {pred}")
    print("-" * 30)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
c:\hate_speech_ukranian_llm\.venv\lib\site-packages\bitsandbytes\autograd\_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Ти повний дурень! Hate-Speech Hate-Speech


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Гарне відео! Neutral Neutral
Ого, яка глибока думка 🤡 Sarcasm Neutral


Compute Accuracy

In [ ]:
import pandas as pd

# Load your results
df = pd.read_csv("results.csv")

# Calculate accuracy: (True/False).mean() gives you the percentage of True
accuracy = df.groupby("stage").apply(
    lambda x: (x["gold"] == x["pred"]).mean(), include_groups=False
)

print("--- Accuracy by Stage ---")
print(accuracy)

stage
zero-shot    0.666667
dtype: float64


C:\Users\avror\AppData\Local\Temp\ipykernel_1104\3867451120.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  acc = df.groupby("stage").apply(lambda x: (x.gold == x.pred).mean())


LoRA Setup

In [ ]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

Training Arguments & Trainer

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    fp16=True,
    output_dir="./results",
    save_strategy="epoch",
    evaluation_strategy="no",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    max_seq_length=512,
    args=training_args,
)

Start Training

In [ ]:
trainer.train()